In [1]:
#train3 from 2: use adam, remove weight_decay; one BW channel (not 3)
import torch,os 
import torch.nn as nn
import torch.optim as optim

from torchvision.transforms import v2
import torchvision.models as models
import pandas as pd
import matplotlib.pyplot as plt

# from models import *
from utils import progress_bar
from tqdm import tqdm
from configparser import ConfigParser
from torch.utils.data import  DataLoader
from LIDC_data import LIDC_Dataset

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

prep_tr = [
    v2.Lambda(lambda x: torch.clamp(x,-1000,400)),
    v2.Lambda(lambda x: (x+1000)/1400),
    v2.CenterCrop((384,384)),
    # v2.Lambda(lambda x: x.expand(3,-1,-1))
]
aug_tr = [
    v2.RandomAffine(degrees=10),
    v2.RandomHorizontalFlip(),
    # v2.GaussianNoise(0,0.1)
]
trans_train = v2.Compose( prep_tr + aug_tr )
trans_test = v2.Compose( prep_tr  )

In [3]:

parser = ConfigParser()
parser.read('.settings')
root_dir = parser.get('dataset','root_dir') #/workspaces/data/lidc-idri/slices
meta_dir = parser.get('dataset','meta_dir') #/workspaces/data/lidc-idri/splits
train_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'train_malB.csv'),transform=trans_train)
test_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'test_malB.csv'),transform=trans_test)
total_train_data = len(train_data)
total_test_data = len(test_data)
print('total_train_data:',total_train_data, 'total_test_data:',total_test_data)

trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16)

total_train_data: 5495 total_test_data: 2354


In [4]:
# net = ResNet18(num_classes=2)
net = models.resnet18(num_classes=2)
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# net.fc = nn.Linear(net.fc.in_features, 2)
net = net.to(device)

In [5]:
lr = 1e-4
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

trainning_accuracy=[]
trainning_loss=[]
testing_accuracy=[]
testing_loss=[]

In [6]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader,desc=f"[Epoch {epoch+1}]")):
        inputs, targets = inputs.to(device), targets.to(device)
        # targets = targets.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = net(inputs)
        
        loss = criterion(outputs, targets)
        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        # preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
        
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        # correct += (preds == targets.long()).sum().item()

    train_acc = 100.*correct/total
    train_loss = train_loss/(batch_idx+1)
    print(f"Train Loss: {train_loss}, Train Acc: {train_acc:.2f}%")
    trainning_accuracy.append(train_acc)
    trainning_loss.append( train_loss )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            # targets = targets.float().unsqueeze(1)
            outputs = net(inputs)
            
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            # preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            # correct += (preds == targets.long()).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_acc = 100.*correct/total
        test_loss = test_loss/(batch_idx+1)
        print(f"Test Loss: {test_loss}, Test Acc: {test_acc:.2f}%")
        testing_accuracy.append(100.*correct/total)
        testing_loss.append(test_loss/(batch_idx+1))
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_owndata.pth')
        best_acc = acc

In [7]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
 
 #   scheduler.step()

[Epoch 1]:   0%|          | 0/344 [00:00<?, ?it/s]

[Epoch 1]: 100%|██████████| 344/344 [01:22<00:00,  4.16it/s]


Train Loss: 0.6889607020935347, Train Acc: 55.89%
 [=================================================>]  Step: 33ms | Tot: 20s337ms | Loss: 0.739 | Acc: 58.199% (1370/2354 148/148  
Test Loss: 0.7388321956669962, Test Acc: 58.20%
Saving..


[Epoch 2]: 100%|██████████| 344/344 [01:16<00:00,  4.48it/s]


Train Loss: 0.6747313814107762, Train Acc: 58.40%
 [=================================================>]  Step: 21ms | Tot: 16s671ms | Loss: 0.711 | Acc: 53.483% (1259/2354 148/148  
Test Loss: 0.7106303246440114, Test Acc: 53.48%


...

Train Loss: 0.14258936319911722, Train Acc: 94.50%
 [=================================================>]  Step: 26ms | Tot: 16s331ms | Loss: 0.507 | Acc: 83.985% (1977/2354 148/148  
Test Loss: 0.506982616340817, Test Acc: 83.98%
Saving..


[Epoch 49]: 100%|██████████| 344/344 [01:15<00:00,  4.54it/s]


Train Loss: 0.1352925376051517, Train Acc: 95.16%
 [=================================================>]  Step: 20ms | Tot: 16s512ms | Loss: 0.505 | Acc: 84.410% (1987/2354 148/148  
Test Loss: 0.5050809810618867, Test Acc: 84.41%
Saving..


[Epoch 50]: 100%|██████████| 344/344 [01:16<00:00,  4.52it/s]


Train Loss: 0.1297230873024091, Train Acc: 95.47%
 [=================================================>]  Step: 20ms | Tot: 16s458ms | Loss: 0.632 | Acc: 80.671% (1899/2354 148/148  
Test Loss: 0.6315658956489248, Test Acc: 80.67%


In [8]:
net

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
from torchsummary import summary
summary(net, (1, 384, 384))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 192, 192]           3,136
       BatchNorm2d-2         [-1, 64, 192, 192]             128
              ReLU-3         [-1, 64, 192, 192]               0
         MaxPool2d-4           [-1, 64, 96, 96]               0
            Conv2d-5           [-1, 64, 96, 96]          36,864
       BatchNorm2d-6           [-1, 64, 96, 96]             128
              ReLU-7           [-1, 64, 96, 96]               0
            Conv2d-8           [-1, 64, 96, 96]          36,864
       BatchNorm2d-9           [-1, 64, 96, 96]             128
             ReLU-10           [-1, 64, 96, 96]               0
       BasicBlock-11           [-1, 64, 96, 96]               0
           Conv2d-12           [-1, 64, 96, 96]          36,864
      BatchNorm2d-13           [-1, 64, 96, 96]             128
             ReLU-14           [-1, 64,